In [12]:
import parameter
import argparse
import sys
import torch
import os
from plyfile import PlyData
import numpy as np
import cv2

from utils.colmap_utils import (
    read_extrinsics_binary, 
    read_intrinsics_binary, 
    read_points3D_binary, 
    read_points3D_text,
    qvec2rotmat
)

def ParamReader(argv=None):
    if argv is None:
        argv = sys.argv[1:]
    
    parser = argparse.ArgumentParser()
    data = parameter.data
    for key, value in data.items():
        parser.add_argument(f'--{key}', default=value, help=f'{key} parameter')
    
    parser.add_argument("--detect_anomaly", action="store_true", default=False)
    parser.add_argument("--test_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])
    parser.add_argument("--save_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])    
    parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])
    parser.add_argument("--start_checkpoint", type=str, default=None)
    
    args = parser.parse_args(argv)
    return args

class PointClond:
    def __init__(self, positions, colors, normals):
        self.positions = positions
        self.colors = colors
        self.normals = normals

class ImageInfo:
    def __init__(self,image_name, image_path, image, image_width, image_height, R, T, fov_x, fov_y, device):
        self.image_name=image_name
        self.image_path=image_path
        self.image = torch.from_numpy(image).to(device)
        self.image_width=image_width
        self.image_height=image_height
        self.R=R
        self.T=T
        self.fov_x=fov_x
        self.fov_y=fov_y
        self.zfar = 100.0  # 远平面
        self.znear = 0.01  # 近平面
        # 计算相机矩阵并移动到指定设备
        self.viewMatrix = self.getViewMatrix(self.R, self.T).to(device)
        # 计算投影矩阵并移动到指定设备
        self.projMatrix = self.getProjMatrix(self.znear, self.zfar, self.fov_x, self.fov_y).to(device)
              
class GSDataLoader:
    def __init__(self,data_path,reading_dir,device):
        self.data_path=data_path
        self.reading_dir=reading_dir
        self.device=device
        self.cameras=[]#相机
        self.points={}#点云
        self.data_path=r'D:\CODE_ALL\3DGS\data' 
        self.loadColmap(reading_dir)
    
    def loadColmap(self,reading_dir):
        #读取相机的内参
        cameras_intrinsic_path = os.path.join( self.data_path, "sparse", "0", "cameras.bin")
        cam_intrinsics = read_intrinsics_binary(cameras_intrinsic_path)
        #读取相机的外参
        cameras_extrinsic_path= os.path.join( self.data_path, "sparse", "0", "images.bin")
        cam_extrinsics = read_extrinsics_binary(cameras_extrinsic_path)
        #print('外参',cam_extrinsics)
        #print('内参',cam_intrinsics)
        
        #点云
        ply_path = os.path.join(self.data_path,  "sparse", "0", "points3D.ply")
        plydata = PlyData.read(ply_path)
        # 提取顶点数据
        vertices = plydata['vertex']
        x = vertices['x']
        y = vertices['y']
        z = vertices['z']
        positions = np.vstack([x,y, z]).T
        #print( positions)
        
        
        # 提取颜色数据
        red = vertices['red']
        green = vertices['green']
        blue = vertices['blue']
        colors = np.vstack([red,green, blue]).T/255.0
        #print( colors)
        
        # 提取法线数据
        nx = vertices['nx']
        ny = vertices['ny']
        nz = vertices['nz']
        normals = np.vstack([nx,ny, nz]).T
        #print( normals)
        self.points=PointClond(positions,colors,normals)
        
        # 读取图片
        # 将图像文件夹路径与数据根路径结合，形成完整的图像文件夹路径
        images_folder=os.path.join(self.data_path, reading_dir)
        
        for _, image_info  in cam_extrinsics.items():
            intrinsic=cam_intrinsics[image_info.camera_id]
             # 获取图像的高度和宽度
            image_height = intrinsic.height
            image_width = intrinsic.width
            # 获取图像路径并读取图像
            image_path = os.path.join(images_folder, image_info.name)
            image = cv2.imread(image_path)
            print(image)
            
            
            
            
            
            
        
        
        
    
# 测试
argv = ['--detect_anomaly', '--test_iterations', '100', '2000', '--save_iterations', '100', '2000']
args=ParamReader(argv)
device = torch.device('cpu')
data = GSDataLoader(args.source_path, args.images, device)


ModuleNotFoundError: No module named 'cv2'